In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Reading Dataset

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/EFQM/5-Multi label classification/Multivariate_pollution.csv')
ds.head()

Check target values

In [ ]:
print('snow data', ds['snow'].unique())
print('rain data', ds['rain'].unique())

Checking all data

In [ ]:
ds.info()

Creating time features

In [ ]:
ds['date'] = pd.to_datetime(ds['date'])
ds['year'] = ds['date'].dt.year
ds['month'] = ds['date'].dt.month
ds['day'] = ds['date'].dt.day
ds['hour'] = ds['date'].dt.hour


ds.drop(['date'], axis=1, inplace=True)
ds.head()